In [75]:
import os
import uuid
import pickle

import pandas as pd

import mlflow
from io import BytesIO

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [76]:
from dotenv import load_dotenv
# Carregar as variáveis de ambiente do arquivo .secrets
load_dotenv('.secrets')

RUN_ID = os.getenv('RUN_ID')

# logged_model = f'runs:/{RUN_ID}/model'

# Definir a configuração do backend store do MLflow
os.environ['MLFLOW_S3_ENDPOINT_URL'] = os.getenv('minio_endpoint_url')
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('minio_access_key')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('minio_secret_key')

In [77]:
import boto3

# Criação do cliente S3 apontando para o MinIO
s3_client = boto3.client('s3',
                         endpoint_url=os.environ['MLFLOW_S3_ENDPOINT_URL'],
                         aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
                         aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'])

In [78]:
year = 2021
month = 3
taxi_type = 'green'

bucket_name = 'test-minio'
in_file_name = f'{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'

out_path = f'output/{taxi_type}'
out_file_name = f'{year:04d}-{month:02d}.parquet'

input_file = f'https://s3.amazonaws.com/nyc-tlc/trip+data/{in_file_name}'

RUN_ID = os.getenv('RUN_ID', '874e7ae01334406590ac62ddc0422882')

In [79]:
response = s3_client.get_object(Bucket=bucket_name, Key=in_file_name)
parquet_file = response['Body'].read()
input_file = BytesIO(parquet_file)

In [80]:
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['ride_id'] = generate_uuids(len(df))

    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [81]:
def load_model(run_id):
    logged_model = f's3://mlflow-models/1/{RUN_ID}/artifacts/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model


def apply_model(input_file, run_id, out_path, out_file_name):

    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    
    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id
    
    os.makedirs(out_path, exist_ok=True)

    df_result.to_parquet(os.path.join(out_path, out_file_name), index=False)

In [82]:
apply_model(input_file=input_file, run_id=RUN_ID, out_path=out_path, out_file_name=out_file_name)

In [83]:
!ls output/green/

2021-02.parquet  2021-03.parquet
